## 1. Initial setting

Mitsuba 3 does not use external array-based numeric backend such as NumPy or PyTorch, but use its own backend, named Dr. Jit.
You should call `mi.set_variant` method before running any other mitsuba functions.

Mitsuba 3 can load scene information using XML files or Python dictionary variables. To testing Mitsuba 3 API usage, `mi.cornell_box()` function is useful.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import drjit as dr
import mitsuba as mi
mi.set_variant('cuda_ad_rgb')
# mi.set_variant('scalar_rgb')

scene = mi.load_dict(mi.cornell_box())

In [12]:
print(f"{type(scene) = }")
print(scene)

type(scene) = <class 'mitsuba.cuda_ad_rgb.Scene'>
Scene[
  children = [
    Rectangle[
      to_world = [[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, -1],
                  [0, 0, 0, 1]],
      frame = Frame[
        s = [[2, 0, 0]],
        t = [[0, 2, 0]],
        n = [[0, 0, 1]]
      ],
      surface_area = [4],
      bsdf = SmoothDiffuse[
        reflectance = SRGBReflectanceSpectrum[
          value = [[0.885809, 0.698859, 0.666422]]
        ]
      ]
    ],
    Rectangle[
      to_world = [[1, 0, 0, 0],
                  [0, -4.37114e-08, -1, 1],
                  [0, 1, -4.37114e-08, 0],
                  [0, 0, 0, 1]],
      frame = Frame[
        s = [[2, 0, 0]],
        t = [[0, -8.74228e-08, 2]],
        n = [[0, -1, -4.37114e-08]]
      ],
      surface_area = [4],
      bsdf = SmoothDiffuse[
        reflectance = SRGBReflectanceSpectrum[
          value = [[0.885809, 0.698859, 0.666422]]
        ]
      ]
    ],
    Rectangle[
      to_world =

-----
## 2. Ray intersection and little bit about Dr. Jit 

In [90]:
o = np.array([[0, 0, 0], [1, 1, 1], [0, 0, 0], [0, 0, 1]])
d = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0]])
ray = mi.Ray3f(o, d)
print(f"{type(ray) = }")
print(f"{dr.shape(ray) = }")
print(f"{dr.width(ray) = }")
print(ray, '\n')

print(f"{type(ray.o) = }")
print(f"{dr.shape(ray.o) = }")
print(ray.o)
print(f"{type(ray.d) = }")
print(f"{dr.shape(ray.d) = }")
print(ray.d)

type(ray) = <class 'mitsuba.cuda_ad_rgb.Ray3f'>
dr.shape(ray) = None
dr.width(ray) = 4
Ray3fDC[
  o = [[0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 1]],
  d = [[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0]],
  maxt = [3.40282e+38],
  time = [0],
] 

type(ray.o) = <class 'mitsuba.cuda_ad_rgb.Point3f'>
dr.shape(ray.o) = [3, 4]
[[0.0, 0.0, 0.0],
 [1.0, 1.0, 1.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0]]
type(ray.d) = <class 'mitsuba.cuda_ad_rgb.Vector3f'>
dr.shape(ray.d) = [3, 4]
[[0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0]]


In [77]:
o = np.zeros((2,3,4))
d = np.ones((2,3,4))
try:
    ray = mi.Ray3f(o, d)
except:
    print("Error!")

o = np.zeros((2,4,3))
d = np.ones((2,4,3))
try:
    ray = mi.Ray3f(o, d)
except:
    print("Error!")

o = np.zeros((3,2,4))
d = np.ones((3,2,4))
try:
    ray = mi.Ray3f(o, d)
except:
    print("Error!")

Error!
Error!
Error!


In [62]:
p =mi.Point3f(np.zeros((2,3)))
print(f"{type(p) = }")
print(f"{dr.shape(p) = }")
print(p)
print(np.array_equal(p.numpy(), np.array(p)))
print(f"{p.numpy().shape = }")

print(f"{mi.Point3f(np.zeros((3,))).numpy().shape = }")

type(p) = <class 'mitsuba.cuda_ad_rgb.Point3f'>
dr.shape(p) = [3, 2]
[[0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]]
True
p.numpy().shape = (2, 3)
mi.Point3f(np.zeros((3,))).numpy().shape = (1, 3)


In [73]:
p = mi.Point3f(np.arange(6).reshape(2,3))
print(f"{dr.width(p)}")
print(f"{p}", '\n')

print(f"{dr.shape(p[0]) = }")
print(f"{p[0] = }", '\n')

print(f"{dr.shape(p[0,:]) = }")
print(f"{p[0,:] = }", '\n')

print(f"{dr.shape(p.x) = }")
print(f"{p.x = }", '\n')

try:
    print(f"{p[:,0] = }")
except:
    print("Error: p[:,0]")


2
[[0.0, 1.0, 2.0],
 [3.0, 4.0, 5.0]] 

dr.shape(p[0]) = [2]
p[0] = [0.0, 3.0] 

dr.shape(p[0,:]) = [2]
p[0,:] = [0.0, 3.0] 

dr.shape(p.x) = [2]
p.x = [0.0, 3.0] 

Error: p[:,0]


In [91]:
si = scene.ray_intersect(ray)
print(f"{type(si) = }")
print(f"{dr.shape(si) = }")
print(f"{dr.width(si) = }")
print(si)


type(si) = <class 'mitsuba.cuda_ad_rgb.SurfaceInteraction3f'>
dr.shape(si) = None
dr.width(si) = 4
SurfaceInteraction[
  t = [inf, inf, 1, 1],
  time = [0],
  wavelengths = [],
  p = [[0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 1]],
  shape = [0x0, 0x0, 0x250357cd770, 0x250357cd770],
  uv = [[0, 0],
        [0, 0],
        [0.5, 0.5],
        [1, 0.5]],
  n = [[0, 0, 0],
       [0, 0, 0],
       [-1, 0, -4.37114e-08],
       [-1, 0, -4.37114e-08]],
  sh_frame = Frame[
    s = [[1, -0, -0],
         [1, -0, -0],
         [-4.37114e-08, 0, 1],
         [-4.37114e-08, 0, 1]],
    t = [[0, 0, -0],
         [0, 0, -0],
         [0, 1, 0],
         [0, 1, 0]],
    n = [[0, 0, 0],
         [0, 0, 0],
         [-1, 0, -4.37114e-08],
         [-1, 0, -4.37114e-08]]
  ],
  dp_du = [[0, 0, 0],
           [0, 0, 0],
           [-8.74228e-08, 0, 2],
           [-8.74228e-08, 0, 2]],
  dp_dv = [[0, 0, 0],
           [0, 0, 0],
           [0, 2, 0],
           [0, 2, 0]],
  dn_du = [

In [94]:
import mitsuba as mi
mi.set_variant('scalar_rgb')
print(mi.Point3f(1, 2, 3).numpy().shape)

mi.set_variant('cuda_ad_rgb')
print(mi.Point3f(1, 2, 3).numpy().shape)

mi.set_variant('llvm_ad_rgb')
print(mi.Point3f(1, 2, 3).numpy().shape)

(3,)
(1, 3)
(1, 3)
